In [ ]:
#| output: false
using CSV
using DataFrames
using DataFramesMeta
using Distributions
using Interpolations
using Plots
using StatsPlots
using Unitful

Plots.default(; margin=6Plots.mm)

include("depthdamage.jl")

# Depth damage function


In [ ]:
haz_fl_dept = CSV.read("data/haz_fl_dept.csv", DataFrame) # read in the file
desc = "two story, no basement, Structure"
row = @rsubset(haz_fl_dept, :Description == desc)[1, :] # select the row I want
dd = DepthDamageData(row) # extract the depth-damage data
damage_fn = get_depth_damage_function(dd.depths, dd.damages) # get the depth-damage function

# Single year function


In [ ]:
function single_year_cost_benefit(flood_dist, damage_fn, elevation_cost, house_area, house_value, Δh)

    # calculate the expected damages
offset = 1.66 
house_dist = GeneralizedExtremeValue(flood_dist.μ - offset, flood_dist.σ, flood_dist.ξ)
samples = rand(house_dist, 100_000) .* 1u"ft"
damages = damage_fn.(samples)
expected_damages_pct = mean(damages)
expected_damages_usd = house_value * expected_damages_pct / 100
c_dmg = expected_damages_usd

    # calculate the cost of elevating
Δh = Δh .* 1u"ft"
c_constr = elevation_cost.(Δh, house_area)

    # return the total cost and benefit
    return -c_constr - c_dmg
end

# NPV function


In [ ]:
function npv_cost_benefit(flood_dist, damage_fn, elevation_cost, house_area, house_value, Δh, T, discount_rate)
    # calculate the costs and benefits for each year, and then discount
    # see above!
    cost_benefits = zeros(T)
    for i in 1:T
        if i== 1
            cost_benefits[i] = single_year_cost_benefit(flood_dist, damage_fn, elevation_cost, house_area, house_value, Δh)
        else
            cost_benefits[i] = single_year_cost_benefit(flood_dist, damage_fn, elevation_cost, house_area, house_value, 0)
        end
    end
    npv = sum(cost_benefits .* (1 - discount_rate) .^ (0:9))
    return npv
end

# One SOW, several actions


In [ ]:
flood_dist = GeneralizedExtremeValue(5, 1, 0.1) 

elevation_cost = get_elevation_cost_function()
house_area = 1000u"ft^2"
house_value = 400000
h = 5
T = 10
discount_rate = 0.05

npv_cost_benefit(flood_dist, damage_fn, elevation_cost, house_area, house_value, Δh, T, discount_rate)

In [ ]:
house_area = 1000u"ft^2"
house_value = 400000
h = 10
T = 10
discount_rate = 0.05
npv_cost_benefit(flood_dist, damage_fn, elevation_cost, house_area, house_value, h, T, discount_rate) 

In [ ]:
house_area = 1000u"ft^2"
house_value = 400000
h = 0
T = 10
discount_rate = 0.05
npv_cost_benefit(flood_dist, damage_fn, elevation_cost, house_area, house_value, h, T, discount_rate) 

# Sensitivity test

# Discussion
